In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Read in the data
lats = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/lons.npy')

p_ctl = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/ctl/slp_11-16_12.npy')
p_box = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_cape_box/slp_11-16_12.npy')
p_hole = np.load('/p/work1/lloveras/real/nov2018/proc_oct/4km_files/adj_cape_hole/slp_11-16_12.npy')

dp_box = p_box - p_ctl
dp_hole = p_hole - p_ctl


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*1.75

### Plot subdomain
min_lat = 31
max_lat = 49
min_lon = -79
max_lon = -61

### Cartopy projection
proj = crs.PlateCarree()

fig, axd = plt.subplot_mosaic([['left','right'],
                               ['cbar','cbar']],
                              constrained_layout=True, figsize=(7.2,4.2), dpi=2000, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':proj},
                                              'right':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['left'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

cs1 = axd['left'].contour(lons, lats, smooth2d(p_ctl,20),
                  levels=np.arange(0,1500,6),
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='k')
cs1 = axd['left'].contour(lons, lats, smooth2d(p_box,20),
                  levels=np.arange(0,1500,6),
                  colors='magenta', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='magenta')
im1 = axd['left'].contourf(lons, lats, dp_box, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['left'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('CAPE Box')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
##############

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['right'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-75, -70, -65])
gls.ylocator = mticker.FixedLocator([35, 40, 45])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['right'].contour(lons, lats, smooth2d(p_ctl,20),
                  levels=np.arange(0,1500,6),
                  colors='k', transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='k')
cs1 = axd['right'].contour(lons, lats, smooth2d(p_hole,20),
                  levels=np.arange(0,1500,6),
                  colors='magenta', transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,6),fontsize=6,colors='magenta')
im1 = axd['right'].contourf(lons, lats, dp_hole, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['right'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['right'].set_title('CAPE Hole')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/4km_cape_slp.pdf',bbox_inches='tight')
plt.show()
